# Тестирование модуля обратной задачи ЭЭГ с JuliaCall

Этот ноутбук демонстрирует использование модуля InverseNpde для решения обратной задачи ЭЭГ
через современный JuliaCall интерфейс.

Включает:
- Базовый пример использования
- Различные конфигурации нейронных сетей
- Сравнение производительности
- Визуализацию результатов

In [ ]:
# Импорт необходимых библиотек

import sys
import os
sys.path.append('../src/python')

import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

# Проверяем доступность JuliaCall интерфейса
try:
    from julia_interface import EEGInverseSolver, create_fast_solver, create_gpu_solver
    print("✓ JuliaCall интерфейс доступен")
except ImportError as e:
    print(f"❌ JuliaCall интерфейс недоступен: {e}")
    print("Убедитесь, что JuliaCall установлен: pip install juliacall")
    
# Настройка matplotlib
plt.style.use('default')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12

## 1. Базовый пример использования

Создаем простой решатель с конфигурацией по умолчанию и запускаем эксперимент.

In [ ]:
# Создаем решатель с быстрой конфигурацией для тестирования
print("🚀 Создание быстрого решателя...")
solver = create_fast_solver()

# Показываем конфигурацию
config = solver.get_config_summary()
print("\n📋 Конфигурация решателя:")
print(f"Нейронная сеть: {config['neural_config']['hidden_layers']} слоев")
print(f"Оптимизатор: {config['optimization_config']['optimizer']}")
print(f"Итерации: {config['optimization_config']['max_iterations']}")
print(f"Область: x∈{config['domain']['x_range']}, t∈{config['domain']['t_range']}")

In [ ]:
# Запускаем эксперимент
print("\n🔄 Запуск эксперимента обратной задачи ЭЭГ...")
try:
    results = solver.solve(save_results=False)
    
    # Показываем результаты
    print("✓ Эксперимент завершен успешно!")
    print(f"Финальная ошибка: {results['metrics']['final_loss']:.6f}")
    print(f"MSE: {results['metrics']['mse']:.6f}")
    print(f"MAE: {results['metrics']['mae']:.6f}")
    print(f"Максимальная ошибка: {results['metrics']['max_error']:.6f}")
    
except Exception as e:
    print(f"❌ Ошибка во время эксперимента: {e}")
    print("Проверьте, что JuliaCall и все зависимости установлены правильно")

## 2. Визуализация результатов

Создаем графики для сравнения аналитического и предсказанного решений.

In [ ]:
if 'results' in locals() and results['solved']:
    print("📊 Создание визуализации...")
    
    # Подготавливаем данные
    x_range = results['x_range']
    y_range = results['y_range']
    analytic_sol = results['analytic_solution']
    predicted_sol = results['predicted_solution']
    
    # Преобразуем в 2D сетки
    X, Y = np.meshgrid(x_range, y_range)
    analytic_2d = analytic_sol.reshape(len(x_range), len(y_range))
    predicted_2d = predicted_sol.reshape(len(x_range), len(y_range))
    error = np.abs(analytic_sol - predicted_sol).reshape(len(x_range), len(y_range))
    
    # Создаем графики
    fig, axes = plt.subplots(1, 3, figsize=(18, 5))
    
    # Аналитическое решение
    im1 = axes[0].contourf(X, Y, analytic_2d, levels=20, cmap='viridis')
    axes[0].set_title('Аналитическое решение φ(x,y)')
    axes[0].set_xlabel('x')
    axes[0].set_ylabel('y')
    plt.colorbar(im1, ax=axes[0])
    
    # Предсказанное решение
    im2 = axes[1].contourf(X, Y, predicted_2d, levels=20, cmap='viridis')
    axes[1].set_title('Предсказанное решение φ(x,y)')
    axes[1].set_xlabel('x')
    axes[1].set_ylabel('y')
    plt.colorbar(im2, ax=axes[1])
    
    # Ошибка
    im3 = axes[2].contourf(X, Y, error, levels=20, cmap='Reds')
    axes[2].set_title('Ошибка |φ_аналитич - φ_предсказ|')
    axes[2].set_xlabel('x')
    axes[2].set_ylabel('y')
    plt.colorbar(im3, ax=axes[2])
    
    plt.tight_layout()
    plt.show()
    
    print("✓ Визуализация создана")
else:
    print("⚠️ Нет результатов для визуализации")

## 3. Получение источников сигнала

Демонстрируем, как получить источники (плотность заряда и тока) в заданных точках.

In [ ]:
if 'results' in locals() and results['solved']:
    print("⚡ Получение источников сигнала...")
    
    # Задаем точки для анализа
    x_points = [0.0, 1.0, -1.0, 2.0]
    y_points = [0.0, 0.0, 0.0, 0.0]
    z_points = [0.0, 0.0, 0.0, 0.0]
    
    try:
        sources = solver.get_sources(x_points, y_points, z_points)
        
        print(f"Точки: x={x_points}, y={y_points}, z={z_points}")
        print(f"Плотность заряда ρ: {sources['rho']}")
        print(f"Плотность тока jx: {sources['jx']}")
        print(f"Плотность тока jy: {sources['jy']}")
        print(f"Плотность тока jz: {sources['jz']}")
        
        # Создаем график источников
        fig, axes = plt.subplots(2, 2, figsize=(12, 10))
        
        # Плотность заряда
        axes[0,0].bar(range(len(x_points)), sources['rho'])
        axes[0,0].set_title('Плотность заряда ρ')
        axes[0,0].set_xlabel('Точка')
        axes[0,0].set_ylabel('ρ')
        
        # Компоненты плотности тока
        axes[0,1].bar(range(len(x_points)), sources['jx'], label='jx')
        axes[0,1].bar(range(len(x_points)), sources['jy'], alpha=0.7, label='jy')
        axes[0,1].bar(range(len(x_points)), sources['jz'], alpha=0.5, label='jz')
        axes[0,1].set_title('Плотность тока j')
        axes[0,1].set_xlabel('Точка')
        axes[0,1].set_ylabel('j')
        axes[0,1].legend()
        
        # 3D визуализация вектора тока
        from mpl_toolkits.mplot3d import Axes3D
        ax = fig.add_subplot(224, projection='3d')
        ax.quiver(x_points, y_points, z_points, 
                 sources['jx'], sources['jy'], sources['jz'],
                 length=0.1, normalize=True)
        ax.set_title('Вектор плотности тока j')
        ax.set_xlabel('x')
        ax.set_ylabel('y')
        ax.set_zlabel('z')
        
        plt.tight_layout()
        plt.show()
        
        print("✓ Источники получены и визуализированы")
        
    except Exception as e:
        print(f"❌ Ошибка получения источников: {e}")
else:
    print("⚠️ Сначала запустите эксперимент")

## 4. Восстановление сигнала по источникам

Показываем, как восстановить скалярный потенциал по известным источникам.

In [ ]:
if 'results' in locals() and results['solved']:
    print("🔄 Восстановление сигнала по источникам...")
    
    try:
        # Создаем сетку точек для восстановления
        x_test = np.linspace(-2, 2, 10)
        y_test = np.linspace(-2, 2, 10)
        z_test = np.zeros(10)
        
        # Восстанавливаем сигнал
        recovered_signal = solver.predict_signal(
            {'rho': np.ones(10), 'jx': np.zeros(10), 'jy': np.zeros(10), 'jz': np.zeros(10)},
            x_test, y_test, z_test
        )
        
        print(f"Восстановленный сигнал в точках: {recovered_signal}")
        
        # Сравниваем с аналитическим решением
        analytic_signal = [(t + 1)**2 / np.sqrt(x**2 + y**2 + z**2) 
                          for x, y, z in zip(x_test, y_test, z_test)]
        
        # График сравнения
        plt.figure(figsize=(10, 6))
        plt.plot(x_test, recovered_signal, 'o-', label='Восстановленный сигнал', linewidth=2)
        plt.plot(x_test, analytic_signal, 's--', label='Аналитическое решение', linewidth=2)
        plt.xlabel('x')
        plt.ylabel('φ(x, y=0, z=0, t=0)')
        plt.title('Сравнение восстановленного и аналитического сигналов')
        plt.legend()
        plt.grid(True, alpha=0.3)
        plt.show()
        
        print("✓ Сигнал восстановлен и сравнен")
        
    except Exception as e:
        print(f"❌ Ошибка восстановления сигнала: {e}")
else:
    print("⚠️ Сначала запустите эксперимент")

## 5. Бенчмарк различных конфигураций

Сравниваем производительность разных конфигураций нейронных сетей.

In [ ]:
print("🏁 Запуск бенчмарка конфигураций...")

# Определяем конфигурации для сравнения
configs = [
    {
        'name': 'Малая сеть',
        'neural_config': {
            'input_dim': 4,
            'hidden_layers': [16, 16],
            'output_dim': 8,
            'activation': 'σ',
            'use_gpu': False
        },
        'optimization_config': {
            'optimizer': 'adam',
            'learning_rate': 0.01,
            'max_iterations': 200,
            'log_frequency': 25,
            'use_tensorboard': False
        }
    },
    {
        'name': 'Средняя сеть',
        'neural_config': {
            'input_dim': 4,
            'hidden_layers': [32, 32],
            'output_dim': 8,
            'activation': 'σ',
            'use_gpu': False
        },
        'optimization_config': {
            'optimizer': 'adam',
            'learning_rate': 0.005,
            'max_iterations': 300,
            'log_frequency': 30,
            'use_tensorboard': False
        }
    }
]

try:
    benchmark_results = solver.benchmark_configs(configs)
    
    print("\n📊 Результаты бенчмарка:")
    print("-" * 80)
    print(f"{'Конфигурация':<15} {'Успех':<8} {'MSE':<12} {'MAE':<12} {'Финальная ошибка':<15}")
    print("-" * 80)
    
    for result in benchmark_results:
        config_name = configs[result['config_id']-1]['name']
        if result['success']:
            print(f"{config_name:<15} {'✓':<8} {result['metrics']['mse']:<12.6f} {result['metrics']['mae']:<12.6f} {result['metrics']['final_loss']:<15.6f}")
        else:
            print(f"{config_name:<15} {'✗':<8} {'N/A':<12} {'N/A':<12} {'N/A':<15}")
    
    print("-" * 80)
    
    print("✓ Бенчмарк завершен")
    
except Exception as e:
    print(f"❌ Ошибка бенчмарка: {e}")

## 6. Сохранение и загрузка результатов

Демонстрируем сохранение результатов эксперимента в файл и их последующую загрузку.

In [ ]:
if 'results' in locals() and results['solved']:
    print("💾 Сохранение результатов...")
    
    # Создаем директорию для результатов
    results_dir = Path("results")
    results_dir.mkdir(exist_ok=True)
    
    save_path = results_dir / "eeg_test_results.jld2"
    
    try:
        solver.save_results(str(save_path))
        print(f"✓ Результаты сохранены в {save_path}")
        
        # Загружаем результаты обратно
        print("📂 Загрузка результатов...")
        loaded_results = solver.load_results(str(save_path))
        
        # Проверяем, что данные загружены правильно
        print("🔍 Проверка загруженных данных:")
        print(f"MSE: {loaded_results['metrics']['mse']:.6f}")
        print(f"MAE: {loaded_results['metrics']['mae']:.6f}")
        print(f"Размер аналитического решения: {len(loaded_results['analytic_solution'])}")
        print(f"Размер предсказанного решения: {len(loaded_results['predicted_solution'])}")
        
        print("✓ Сохранение и загрузка работают корректно")
        
    except Exception as e:
        print(f"❌ Ошибка сохранения/загрузки: {e}")
else:
    print("⚠️ Сначала запустите эксперимент")

## Заключение

В этом ноутбуке мы продемонстрировали:

1. **Базовое использование** модуля InverseNpde для решения обратной задачи ЭЭГ
2. **Визуализацию результатов** с сравнением аналитических и предсказанных решений
3. **Получение источников сигнала** (плотность заряда и тока) в заданных точках
4. **Восстановление сигнала** по известным источникам
5. **Бенчмарк различных конфигураций** нейронных сетей
6. **Сохранение и загрузку результатов** экспериментов

### Ключевые возможности модуля:

- ✅ Конфигурируемые физические константы
- ✅ Настраиваемая архитектура нейронной сети
- ✅ Различные оптимизаторы и параметры обучения
- ✅ Поддержка GPU для ускорения вычислений
- ✅ Интеграция с TensorBoard для мониторинга
- ✅ Сохранение и загрузка результатов
- ✅ Простой Python API через JuliaCall

### Рекомендации по использованию:

1. Для **быстрых экспериментов** используйте `create_fast_solver()`
2. Для **производительных вычислений** используйте `create_gpu_solver()`
3. Для **исследований** настраивайте конфигурации вручную через `EEGInverseSolver`
4. Всегда сохраняйте результаты для дальнейшего анализа
5. Используйте бенчмарки для выбора оптимальной конфигурации

### Установка JuliaCall:

Для работы с этим модулем необходимо установить JuliaCall:
```bash
pip install juliacall
```

JuliaCall предоставляет современный интерфейс для взаимодействия между Python и Julia, 
заменяя устаревший PyCall.